D'après une étude VIF, il faudrait supprimer les colonnes ZipCode, Latitude, Longitude

In [1]:
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn import model_selection

# Preprocessing

In [2]:
df = pd.read_csv("data_cleaned.csv")
df.drop(['ZipCode','Latitude', 'Longitude'], axis=1, inplace=True)

In [3]:
X = df.drop('TotalGHGEmissions', axis=1)
y = df['TotalGHGEmissions']

In [4]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, y,test_size=0.3)

In [5]:
imputer = KNNImputer(n_neighbors=5)
imputer = imputer.fit(X_train[["ENERGYSTARScore"]])
X_train[["ENERGYSTARScore"]] = imputer.transform(X_train[["ENERGYSTARScore"]])
X_test[["ENERGYSTARScore"]] = imputer.transform(X_test[["ENERGYSTARScore"]])

In [7]:
# MinMaxScaler
rs = MinMaxScaler()
rs = rs.fit(X_train[['CouncilDistrictCode', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'ENERGYSTARScore', 'SteamUse(kBtu)', 'BuildingAge']])
X_train[['CouncilDistrictCode', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'ENERGYSTARScore', 'SteamUse(kBtu)', 'BuildingAge']] = rs.transform(X_train[['CouncilDistrictCode', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'ENERGYSTARScore', 'SteamUse(kBtu)', 'BuildingAge']])
X_test[['CouncilDistrictCode', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'ENERGYSTARScore', 'SteamUse(kBtu)', 'BuildingAge']] = rs.transform(X_test[['CouncilDistrictCode', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'ENERGYSTARScore', 'SteamUse(kBtu)', 'BuildingAge']])

In [9]:
# OneHotEncoder
ohe = OneHotEncoder(sparse=False)
ohe = ohe.fit(X_train[['BuildingType', 'PrimaryPropertyType', 'Neighborhood']])

In [ ]:
ohe_trained = pd.DataFrame(ohe.transform(X_train[['BuildingType', 'PrimaryPropertyType', 'Neighborhood']]), columns=ohe.get_feature_names())
X_train = X_train.drop(columns={'BuildingType', 'PrimaryPropertyType', 'Neighborhood'}, axis = 1)
X_train = X_train.join(ohe_trained)

In [ ]:
ohe_test = pd.DataFrame(ohe.transform(X_test[['BuildingType', 'PrimaryPropertyType', 'Neighborhood']]), columns=ohe.get_feature_names())
X_test = X_test.drop(columns={'BuildingType', 'PrimaryPropertyType', 'Neighborhood'}, axis = 1)
X_test = X_test.join(ohe_test)

# Modélisation